# Chapitre 3 - Modélisation statistique et économétrique des données départementales

In [ ]:
# Paramètre(s) du notebook

# VERBOSE=True
VERBOSE=False

OPTIONS=""
if not VERBOSE:
    OPTIONS="--quiet"


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
from statsmodels.formula.api import ols
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR

In [ ]:
%store -r donnees_2018_hab
%store -r donnees_2018

In [ ]:
donnees_2018_hab=donnees_2018_hab.drop(columns=['REG', 'Libellé','Crim_Del_PN_GN','Crim_Del_GN_hab','Crim_Del_PN_hab'])



# 1 - Approche économétrique

Dans une première approché économétrique, nous essayons de prédire le nombre de crimes à partir uniquement du nombre de boucherie, puis en rajoutant des variables de contrôles

## 1-A Régression linéaire simple

In [ ]:
print("*"*100)
print(""*100)
print("Une régression simple sur l'ensemble des données non transformées :")
print("-"*100)

In [ ]:
# Code from https://towardsdatascience.com/simulating-replicating-r-regression-plot-in-python-using-sklearn-4ee48a15b67

lm = LinearRegression()

X=donnees_2018_hab[[ 'Nb_Boucherie_dep_hab']]
Y=donnees_2018_hab[['Crim_Del_PN_GN_hab']]

lm.fit(X, Y)


# # for predictions
predictions = lm.predict(X)

beta_hat = [lm.intercept_.tolist()] + lm.coef_.tolist()
print("coefficients : ",beta_hat)
plt.scatter(X, Y)
plt.xlabel("Nb_Boucherie_dep_hab")
plt.ylabel("Crim_Del_PN_GN_hab")
plt.title('Nb_Boucherie_dep_hab*Crim_Del_PN_GN_hab')

In [ ]:
reg_with_statsmodels = ols(" Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab", data = donnees_2018_hab).fit()
print(reg_with_statsmodels.summary())

In [ ]:
print(" "*100)
print(" ----- INTERPRETATION : le modèle est invalide par la F-statistique et la non significativité du coefficient associé à Nb_Boucherie_dep_hab   (R² quasi nul) ")
print(" ----- INTERPRETATION : le R² est quasi nul, suggérant la non-corrélation des deux variables, et ainsi un faible pouvoir prédictif ")
print(" ----- INTERPRETATION : on tente de rajouter des régresseurs supplémentaires pour gagner en prédictivité.")

## 1-B Régression linéaire multiple

In [ ]:
print("*"*100)
print(" "*100)
print("Une régression multiple sur l'ensemble des données non transformées :")
print("-"*100)

In [ ]:
# Code from https://towardsdatascience.com/simulating-replicating-r-regression-plot-in-python-using-sklearn-4ee48a15b67
lm = LinearRegression()

X=donnees_2018_hab[['MED18', 'TP6018', 'D118', 'D918', 'RD18', 'T1_2018',
       'Nb_PN_GN_dep_100k_hab', 'Nb_Boucherie_dep_hab']]
Y=donnees_2018_hab[['Crim_Del_PN_GN_hab']]

lm.fit(X, Y)



# # for predictions
predictions = lm.predict(X)
print("variables explicatives",['MED18', 'TP6018', 'D118', 'D918', 'RD18', 'T1_2018',
       'Nb_PN_GN_dep_100k_hab', 'Nb_Boucherie_dep_hab'])
print("coefficients",lm.coef_)
print("constante",lm.intercept_)

print("R² :",sklearn.metrics.r2_score(Y,predictions ))
print(" "*100)
print(" ----- INTERPRETATION : nous obtenons un meilleur pouvoir prédictif par le modèle, notamment liée à la forte corrélation entre le ratio interdécile et la variable à expliquer  ")
print(" ----- INTERPRETATION : nous regarderons ultérieurement les résidus  ")


In [ ]:
print("*"*100)
print(" "*100)
print("Une régression multiple sur l'ensemble des données non transformées part statsmodel :")
print(" Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab + MED18 + TP6018 + D118 + D918 + RD18  + T1_2018+    Nb_PN_GN_dep_100k_hab")
print("-"*100)

formula_reg= " Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab + MED18 + TP6018 + D118 + D918 + RD18  + T1_2018+    Nb_PN_GN_dep_100k_hab"
reg_with_statsmodels = ols( formula_reg, data = donnees_2018_hab).fit()
print(reg_with_statsmodels.summary())


print(" "*100)
print(" ----- INTERPRETATION : on obtient de nouveau le même R² et les mêmes coefficients  ")
print(" ----- INTERPRETATION : Toutes les coefficients sont significatifs sauf Nb_Boucherie_dep_hab et Nb_PN_GN_dep_100k_hab ")
print(" ----- INTERPRETATION : On peut déjà esquisser l'hypothèse qu'il sera difficile de montrer un usage du nombre de boucherie sur le nombre des crimes")
print(" ----- INTERPRETATION : La note [2] nous suggère un problème de multicollinéarité (potentiellement résoluble par une régression PLS)  ou un problème avec les variables numériques")



In [ ]:
print("*"*100)
print(" "*100)
print("Retour sur les résidus :")
print("-"*100)

sns.residplot(predictions.reshape(-1),'Crim_Del_PN_GN_hab', data=donnees_2018_hab,lowess=True,
                                  line_kws={'color': 'red', 'lw': 1, 'alpha': 1})
plt.xlabel("Fitted values")
plt.title('Residual plot')

print(" ----- INTERPRETATION : pour deux départements quasi-similaires en termes socio-économiques, on peut constater un important écart de prédiction de 4000 crimes et délits  ")
print(" ----- INTERPRETATION : l'amplitude d'un résidu est de +/-2000 crimes et délits, sachant que la médiane parmi les départements est aux alentours de 4000 ")

In [ ]:
residuals = donnees_2018_hab["Crim_Del_PN_GN_hab"] - predictions.reshape(-1)
residuals

plt.figure(figsize=(7,7))
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Normal Q-Q Plot")

print(" ----- INTERPRETATION : La normalité des résidus peut être conjecturée si l'on ne considère pas les points atypiques")

In [ ]:
model_norm_residuals_abs_sqrt=np.sqrt(np.abs(residuals))

plt.figure(figsize=(7,7))
sns.regplot(predictions.reshape(-1), model_norm_residuals_abs_sqrt,
              scatter=True,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
plt.ylabel("Standarized residuals")
plt.xlabel("Fitted value")


print(" ----- INTERPRETATION : L'hypothèse d'homoscédasticité ne peut être faite ici.")
print(" ----- INTERPRETATION : En présence d'hétéroscédasticité, l'ouvrage du Woolridge nous suggère d'utiliser le log (ou log(1+y)) pour : ")
print(" ----- INTERPRETATION : * les variables de population ")
print(" ----- INTERPRETATION : * les variables monétaires ")
print(" ----- INTERPRETATION : * les pourcentages")

In [ ]:
print("*"*100)
print(" "*100)
print("Une régression multiple en cross validation avec des données non transformées")
print("-"*100)

scores = cross_val_score(lm, X, Y, scoring='r2', cv=5)
print("R² obtenu en cross validation",scores)

print(" "*100)
print(" ----- INTERPRETATION : En utilisant la validation croisée, nous obtenons plusieurs R² très différents, de quasi-nulle à 0.72")
print(" ----- INTERPRETATION : La forte variabilité du R², compte tenu du faible nombre d'invidividus dans l'échantillon, montre la sensibilité aux outliers")
print(" ----- INTERPRETATION : Ceci présage également la présence de différents groupes hétérogènes comme dans le clustering dans le chapitre 2")
# R² can be negative with small datasets and cv fold https://stackoverflow.com/questions/23036866/scikit-learn-is-returning-coefficient-of-determination-r2-values-less-than-1

## 1-C Régression linéaire multiple avec transformation log

In [ ]:
new_donnees_2018_hab=donnees_2018_hab.copy()
new_donnees_2018_hab['Crim_Del_PN_GN_hab'] = np.log(new_donnees_2018_hab['Crim_Del_PN_GN_hab'])
new_donnees_2018_hab['RD18'] = np.log(new_donnees_2018_hab['RD18'])
new_donnees_2018_hab['D918'] = np.log(new_donnees_2018_hab['D918'])
new_donnees_2018_hab['D118'] = np.log(new_donnees_2018_hab['D118'])
new_donnees_2018_hab['TP6018'] = np.log(new_donnees_2018_hab['TP6018'])
new_donnees_2018_hab['MED18'] = np.log(new_donnees_2018_hab['MED18'])
new_donnees_2018_hab['Nb_Boucherie_dep_hab'] = np.log(new_donnees_2018_hab['Nb_Boucherie_dep_hab'])
new_donnees_2018_hab['Nb_PN_GN_dep_100k_hab'] = np.log(new_donnees_2018_hab['Nb_PN_GN_dep_100k_hab'])
new_donnees_2018_hab['T1_2018'] = np.log(new_donnees_2018_hab['T1_2018'])

formula_reg= " Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab + MED18 + TP6018   + T1_2018 +    Nb_PN_GN_dep_100k_hab"

reg_with_statsmodels = ols( formula_reg, data = new_donnees_2018_hab).fit()
print(reg_with_statsmodels.summary())


## 1-D Régression linéaire multiple avec transformation log et sans les outliers

In [ ]:
new_donnees_2018_hab=donnees_2018_hab[donnees_2018_hab['Crim_Del_PN_GN_hab']<6700]
new_donnees_2018_hab['Crim_Del_PN_GN_hab'] = np.log(new_donnees_2018_hab['Crim_Del_PN_GN_hab'])
new_donnees_2018_hab['RD18'] = np.log(new_donnees_2018_hab['RD18'])
new_donnees_2018_hab['D918'] = np.log(new_donnees_2018_hab['D918'])
new_donnees_2018_hab['D118'] = np.log(new_donnees_2018_hab['D118'])
new_donnees_2018_hab['TP6018'] = np.log(new_donnees_2018_hab['TP6018'])
new_donnees_2018_hab['MED18'] = np.log(new_donnees_2018_hab['MED18'])
new_donnees_2018_hab['Nb_Boucherie_dep_hab'] = np.log(new_donnees_2018_hab['Nb_Boucherie_dep_hab'])
new_donnees_2018_hab['Nb_PN_GN_dep_100k_hab'] = np.log(new_donnees_2018_hab['Nb_PN_GN_dep_100k_hab'])
new_donnees_2018_hab['T1_2018'] = np.log(new_donnees_2018_hab['T1_2018'])

# formula_reg= " Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab + MED18 + TP6018   + T1_2018 +    Nb_PN_GN_dep_100k_hab"
formula_reg= " Crim_Del_PN_GN_hab ~  Nb_Boucherie_dep_hab + Nb_PN_GN_dep_100k_hab  +T1_2018 + TP6018    "
reg_with_statsmodels = ols( formula_reg, data = new_donnees_2018_hab).fit()
print(reg_with_statsmodels.summary())


# 2 - Approche Machine learning

In [ ]:
# https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html
# Both StandardScaler and MinMaxScaler are very sensitive to the presence of outliers.
# MaxAbsScaler therefore also suffers from the presence of large outliers.



# df_train, df_test = train_test_split(donnees_2018_hab, 
#                                      train_size = 0.7, 
#                                      test_size = 0.3
#                                     )



# pipeline = make_pipeline(RobustScaler(),  SVR(kernel='linear',C=1.0, epsilon=0.2))
# pipeline.fit(X_train, y_train)
# # vA = GridSearchCV(pipeline, param_grid=param_grid,
# #                      scoring='roc_auc', cv=10, refit=True)
# # vA.fit(X_train, y_train)

# print(pipeline.predict(X_test))
# print(y_test)




# new_X_train = X_train['Nb_Boucherie_dep_hab']
# new_Y_train = y_train
# new_X_test= X_test['Nb_Boucherie_dep_hab']
# new_Y_test = y_test

# sc_X = RobustScaler()
# sc_Y = RobustScaler()
# X_train_scaled = sc_X.fit_transform(new_X_train.values.reshape(-1, 1))
# Y_train_scaled = sc_Y.fit_transform(new_Y_train.values.reshape(-1, 1))
# X_test_scaled=sc_X.fit_transform(new_X_test.values.reshape(-1, 1))


# svr = SVR(kernel ='linear')
# svr.fit(X_train_scaled, Y_train_scaled)
# plt.scatter(X_train_scaled, Y_train_scaled, color = 'blue')
# plt.scatter(X_train_scaled, svr.predict(X_train_scaled), color = 'red')
# plt.title('Crim_Del_PN_GN_hab vs  Nb_Boucherie_dep_hab  (SVR)')
# plt.xlabel('Nb_Boucherie_dep_hab')
# plt.ylabel('Crim_Del_PN_GN_hab')
# plt.show()

# print(sc_Y.inverse_transform(svr.predict(X_test_scaled).reshape(-1,1)))
# print(new_Y_test)

In [ ]:
df_train, df_test = train_test_split(donnees_2018_hab, 
                                     train_size = 0.7, 
                                     test_size = 0.3
                                    )

X=df_train['Nb_Boucherie_dep_hab'].values.reshape(-1,1)
y=df_train['Crim_Del_PN_GN_hab'].values.reshape(-1,1)

X_test=df_test['Nb_Boucherie_dep_hab'].values.reshape(-1,1)
y_test=df_test['Crim_Del_PN_GN_hab'].values.reshape(-1,1)
reg = make_pipeline(RobustScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X, y)

Y_pred=reg.predict(X_test)
print(Y_pred)
print(y_test)
r2 = sklearn.metrics.r2_score(y_test,Y_pred )
print(r2)

In [ ]:
X=df_train[['MED18', 'TP6018', 'D118', 'D918', 'RD18', 'T1_2018',
       'Nb_PN_GN_dep_100k_hab', 'Nb_Boucherie_dep_hab']].values.reshape(-1,8)
y=df_train['Crim_Del_PN_GN_hab'].values.reshape(-1,1)

X_test=df_test[['MED18', 'TP6018', 'D118', 'D918', 'RD18', 'T1_2018',
       'Nb_PN_GN_dep_100k_hab', 'Nb_Boucherie_dep_hab']].values.reshape(-1,8)
y_test=df_test['Crim_Del_PN_GN_hab'].values.reshape(-1,1)
reg = make_pipeline(RobustScaler(),SGDRegressor(max_iter=1000, tol=1e-3))
reg.fit(X, y)

Y_pred=reg.predict(X_test)
print(Y_pred)
print(y_test)
r2 = sklearn.metrics.r2_score(y_test,Y_pred )
print(r2)